# Guide to use the classifiers of the Quantum_Classifiers_BSQ_201 repository

### Run the following cell for basic imports

In [1]:
import numpy as np
import pennylane as qml

## The utils folder

The utils of the repository folder can be very useful to run the quantum classifiers. Its functions will be described here.

### Quantum embeddings

These are the 3 embeddings coded in pennylane given in this repository. Here is how to import them.

In [2]:
from utils.quantum_embeddings import angle_embedding,iqp_embedding,amplitude_embedding

The iqp embedding is only specified for a number of qubits equal to the number of features. It takes the feature vector as an input.

The amplitude embedding wil automatically create a circuit with the superior or equal power of two of the number of features. It takes the feature vector as an input.

The angle embedding takes the number of qubits and rotation gates as a parameter. To use it in the next algorithms, a wrapper like below must be created so that the embedding function only takes the feature vector as a parameter. The choices of the rotation gates are "X", "Y" and "Z". 

In [3]:
num_qubits=4
rotation="X"
def angle_embedding(a):
        return utils.quantum_embeddings.angle_embedding(
            a, num_qubits=num_qubits, rotation=rotation
        )

If an embedding is called by itself, a pennylane circuit created. These signature of embeddings will be the ones used in the classifier classes.

### Ansatz

The utils folder also provides a working ansatz for the VQC algorithm with the *HTRU_2* dataset. To call it, just run this line of code:

In [4]:
from utils.quantum_ansatz import ansatz_circuit

There is also a randomized ansatz using Pennylane's random layers. To use it in the classifiers, like the angle embedding, a wrapper needs to be created. For example:

In [5]:
from utils.quantum_ansatz import ansatz_random_layer

def ansatz(params):
    return ansatz_random_layer(params,num_qubits=8,num_params_per_qubits=4)

### Error functions

Multiple error functions are also provided by the utils folder. They can be used as the function to optimize (cost_function) in the VQC and QCNN algorithms. To know more about those function, read the article in the repository. Here is the complete list.

In [6]:
from utils.error_functions import (
    mean_square_error, 
    normalized_mean_square_error,
    normalized_root_mean_square_error,
)

There is also multiple functions to compare the different accuracies of the classifier. They can not be used in the VQC and QCNN algorithm since they do not satisfy the conditions for a error function. These conditions will be described later.

In [7]:
from utils.error_functions import (
    accuracy,
    recall,
    specifity,
    precision,
    negative_prediction_value,
    balanced_accuracy,
    geometric_mean,
    informedness,
)

### Other functions

There is also three other useful functions that can be found in the utils folder. 

The *get_feature_vectors_and_labels* function allows to read datasets from a file. The path, file name and extensions are all given. Also, the number of rows to skip to access the data is set by default at zero.

*get_good_distribution_of_labels* gives an even distribution of the feature vectors of the dataset of the label 1 and -1. The feature vectors and thei repective lables need to be given to the function.

Finaly, *normalize_feature_vectors* normalizes each feature of the feature vectors so that its minimum value is -1 and its maximum is 1. It is useful to do so to make sure that each parameter starts with the same importance and to make sure that the rotation gate actually mean something with each vector (since they are 2pi periodical). Ot only takes the feature vectors for a parameter.

The are import from the line below. Their usage will be shown directly in the next section of the tutorial.

In [8]:
from utils.utils import (
    get_feature_vectors_and_labels, 
    get_good_distribution_of_labels,
    normalize_feature_vectors,
)

## First step: Load the data

The fist step to run the algorithms is to load a dataset to classify. To do so, we will use the three function described in the previous section. For this tutorial, we will use the *HTRU_2* dataset based on pulsar detection. It is available under the dataset folder in the repository. For the VQC and QCNN algorithm, the labels must be -1 or 1. The function *get_good_distribution_of_labels* only works if those labels ar respected. So, in the code below, there will be an example of changing the 0 labels in the datasets into -1.

Here is how to load a dataset, get a significant sample (not only zeros or one to make sure that the classifiers work correctly) and normalize the features with the utils function described earlier. The data can be loaded in a different way as long as there is a feature vectors matrix and a label array or 1 and -1. These types of inputs will be the ones that will work across all of the algorithms. Only the quantum kernel classifier does not need labels of -1 and 1. They just need to be 2 diffents integers.

In [9]:
from utils.utils import (
    get_feature_vectors_and_labels, 
    get_good_distribution_of_labels,
    normalize_feature_vectors,
)

#Load the dataset
feature_vectors, labels = get_feature_vectors_and_labels(
        "HTRU_2", extension="csv", path="datasets/", rows_to_skip=0
    )

#Change the 0 labels into -1
labels=(2*labels)-1
# Get a good sample of the dataset since it is to big
feature_vectors, labels = get_good_distribution_of_labels(
    feature_vectors, labels, 50
)
# Normalize the feature vectors
feature_vectors = normalize_feature_vectors(feature_vectors)

## Note

The use of the three different classifiers are very similar. The quantum kernel algorithm will be described in detail and will be referenced for the other methods to descibe their functioning.

## The quantum kernel classifier

To use the quantum kernel classifier, the *Quantum_Kernel_Classification* class form the *kernel_method.py* file needs to be called.

First, run this line to import the class.

In [10]:
from kernel_method import Quantum_Kernel_Classification

Then, the object needs an embedding funtion and the number of qubits of that embedding. The embedding needs to be a function with th same number of qubits given to the class. It must create the Pennylane circuit that encodes the feature vector. It can not be a QNODE.
Lets create the objet using the amplitude embedding function from the utils file.

In [11]:
import utils.quantum_embeddings
num_qubits=3

kernel_qml = Quantum_Kernel_Classification(
        utils.quantum_embeddings.amplitude_embedding, num_qubits
    )

An object ready to run the quantum kernel classification is created.

To run the algorithm  with a datset, just call the run function. It needs the feature vectors and their respective labels that are two integers.

The optional argument, training ratio, is fixing the ratio between the number of training vectors to be used from all of them given.

Lets call the run function.

In [12]:
training_ratio=0.8

score, predictions = kernel_qml.run(
        feature_vectors, labels, training_ratio=training_ratio
    )

The result of this algorithm is in two parts.

The score return gives the number of labels that the classifier correcty predicted.

The prediction return is the array of all the predicted labels.

Those reults can then be printed to evaluate the precision of the algorithm.

In [13]:
training_period = int(len(labels) * training_ratio)

print("The score of the kernel: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the kernel:  0.95
The predictions of the labels:  [-1 -1 -1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]
The true value of the labels:  [-1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]


## The quantum variationnal algorithm

To use the quantum kernel classifier, the *VQC_Solver* class form the *vqc_method.py* file needs to be called.

First, run this line to import the class.

In [14]:
from vqc_method import VQC_Solver

Then, the object needs an embedding funtion, the ansatz circuit, the number of parameters in the ansatz and the number of qubits of the two circuits.

Like the embedding function described in the kernel classifier, the ansatz callable is a function creating a pennylane circuit that is to be optimized. It can not be a QNODE. It must also have the same number or less of qubits than the one given to the class. It also has the same number or less of parameters than number given to the class.
Lets create the objet using the amplitude embedding function.

In [15]:
import utils.quantum_embeddings
import utils.quantum_ansatz

num_qubits=8
num_params = 12

vqc = VQC_Solver(
    utils.quantum_embeddings.amplitude_embedding,
    utils.quantum_ansatz.ansatz_circuit,
    num_params,
    num_qubits,
)

An object ready to run the VQC (variationnal quantum classifier) is created.

To run the algorithm  with a datset, the same procedure needs to be folled as the quantum kernel method.
However, the run function also needs the optimizer, the error function to be minimized and the classifier function. Also, the labels must be -1 or 1.

The optimizer given needs to only have two parameters: the cost_function and its parameters to optimize. The result of the optimizer also needs a *x* atribute to give the optimized parameters. In this tutorial, we will crate a wrapper optimization function that calls teh COBYLA method form scipy's minimizer.

The error function has also two paramters (the predicted labels and their real value). It is the error to be minimized by the optimizer. Multiple error function ar given in the utils file as specified earlier. The error function must directly use the expval values of the feature vectors in the circuit for the calculation. It can not use a copy of those results. By default, the mean square error function of the utils folder is used.

So, lets use the run function.

In [16]:
from scipy.optimize import minimize
from utils.error_functions import mean_square_error
from pennylane.optimize import NesterovMomentumOptimizer, AdamOptimizer,GradientDescentOptimizer
training_ratio=0.8
"""
def minimisation(cost_function, params):
        result=minimize(
            cost_function,
            params,
            method="COBYLA",
            options={"tol": 1e-08},
        )
        return result.x
"""
def minimisation(cost_function, original_params):
    result = NesterovMomentumOptimizer()
    new_params = original_params
    for _ in range(100):
        new_params = result.step(cost_function, new_params)
    return new_params
score, predictions = vqc.run(
    feature_vectors, labels, minimisation,error_function=mean_square_error, training_ratio=training_ratio
)

The result of this algorithm is the same as the quantum kernel algorithm. Lest print the results.

In [17]:
training_period = int(len(labels) * training_ratio)

print("The score of the VQC: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the VQC:  0.95
The predictions of the labels:  [-1 -1 -1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]
The true value of the labels:  [-1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]


## The quantum convolutional neural network algorithm

To use the quantum kernel classifier, the *QCNN_Solver* class form the *qcnn_method.py* file needs to be called.

First, run this line to import the class.

In [18]:
from qcnn_method import QCNN_Solver

Then, the object needs an embedding funtion and the number of qubits of this circuit.
Lets create the objet using the amplitude embedding function.

In [19]:
import utils.quantum_embeddings

num_qubits=8
def angle(feature_vectors):
    return utils.quantum_embeddings.angle_embedding(feature_vectors,num_qubits,rotation="Y")

qcnn = QCNN_Solver(utils.quantum_embeddings.iqp_embedding, num_qubits)

An object ready to run the QCNN (quantum convolutionnal neural network) is created.

To run the algorithm  with a dataset, the same procedure as the VQC method needs to be followed (the same condition for the optimizer, error function,labels  and classifier function).

However there is two run function.
Lets first use the normal run function. It uses all of the training parameters at each iterations of the optimization.


So, lets use the run function.

### Run function

In [29]:
from scipy.optimize import minimize
from utils.error_functions import mean_square_error
from pennylane.optimize import NesterovMomentumOptimizer, AdamOptimizer,GradientDescentOptimizer
training_ratio=0.8

"""
def minimisation(cost_function, params):
    result=minimize(
        cost_function,
        params,
        method="COBYLA",
        options={
            "maxiter": 100,
            "rhobeg":150,
        },
    )
    return result.x
"""


def minimisation(cost_function, params):
    result=NesterovMomentumOptimizer(momentum=3)
    new_params=params
    for _ in range(100):
        new_params,fun=result.step_and_cost(cost_function,new_params)
        print(fun)
    return new_params    

score, predictions = qcnn.run(
    feature_vectors,
    labels,
    minimisation,
    error_function=mean_square_error,
    training_ratio=training_ratio,
)

0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884
0.49827279506560884


KeyboardInterrupt: 

The result of this algorithm is the same as the quantum kernel algorithm. Lest print the results.

In [27]:
training_period = int(len(labels) * training_ratio)

print("The score of the normal QCNN: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the normal QCNN:  0.45
The predictions of the labels:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
The true value of the labels:  [-1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]


### run_batched function

The *run_batched* function uses the same parameters as the normal run function. However it also has a *num_batches* parameter. Indeed, this function uses partitions of the training parameters to optimize the parametrized circuit. Instead of getting the reult of all of the training vectors for one optimization of the the optimizer, only a few amount of the data is used.

Each batch of the data is used once so that the training vectors is used one. We clearly have a complexity gain.

To use this function, the optimizer needs to run exactly *num_batches iterations*.

This is how to call the run_batched algotithm.

In [22]:
# Lets start over with a new object with not optimized parameter
import utils.quantum_embeddings
num_qubits=8

qcnn = QCNN_Solver(utils.quantum_embeddings.iqp_embedding, num_qubits)

In [23]:
# To run the function:
from scipy.optimize import minimize
from utils.error_functions import mean_square_error
from pennylane.optimize import NesterovMomentumOptimizer, AdamOptimizer,GradientDescentOptimizer

batches = 8
def minimisation(cost_function, params):
        
        result=minimize(
            cost_function,
            params,
            method="COBYLA",
            options={
                "maxiter": batches, #The number of iteration of the optimizer needs to be the same as the number of batches
            },
        )
        
        return result.x

score, predictions = qcnn.run_batched(
    feature_vectors,
    labels,
    minimisation,
    error_function=mean_square_error,
    num_batches=batches,
    training_ratio=training_ratio,
)

The result of this algorithm is the same as the quantum kernel algorithm. Lest print the results.

In [24]:
training_period = int(len(labels) * training_ratio)

print("The score of the batched QCNN: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the batched QCNN:  0.45
The predictions of the labels:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
The true value of the labels:  [-1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1]


You are now ready to use the various quantum classifiers!